# Lab 3: Word Embeddings and Language Modelling

Adam Ek

In this lab we'll explore constructing *static* word embeddings (i.e. word2vec) and building language models. We'll also evaluate these systems on intermediate tasks, namely word similarity and identifying "good" and "bad" sentences.

* For this we'll use pytorch. Some basic operations that will be useful can be found here: https://jhui.github.io/2018/02/09/PyTorch-Basic-operations
* In general: we are not interested in getting state-of-the-art performance :) focus on the implementation and not results of your model. For this reason, you can use a subset of the dataset: the first 5000-10 000 sentences or so, on linux/mac: ```head -n 10000 inputfile > outputfile```. 
* If possible, use the MLTGpu, it will make everything faster :)

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd

# for gpu, replace "cpu" with "cuda:n" where n is the index of the GPU
device = torch.device('cpu')

# Word2Vec embeddings

In this first part we'll construct a word2vec model which will give us *static* word embeddings (that is, they are fixed after training).

After we've trained our model we will evaluate the embeddings obtained on a word similarity task.

## Formatting data


First we need to load some data, you can download the file on canvas under files/03-lab-data/wiki-corpus.txt. The file contains 50 000 sentences randomly selected from the complete wikipedia. Each line in the file contains one sentence. The sentences are whitespace tokenized.

Your first task is to create a dataset suitable for word2vec. That is, we define some ```window_size``` then iterate over all sentences in the dataset, putting the center word in one field and the context words in another (separate the fields with ```tab```).

For example, the sentece "this is a lab" with ```window size = 4``` will be formatted as:
```
center, context
---------------------
this    is a
is      this a lab
a       this is lab
lab     is a
```

this will be our training examples when training the word2vec model.

[3 marks]

In [2]:
data_path = 'wiki-corpus.txt'
WINDOW_SIZE = 4

def corpus_reader(data_path, window_size):
    with open(data_path, "r") as f:
        content = f.read()
        
    content_list = content.splitlines()[:5000]
    window = int(window_size / 2)
    center_and_context = {
        "centre": [],
        "context": [],
    }
        
    for sentence in content_list:
        sentence = sentence.split(' ')
        
        for i in range(len(sentence)):
            before = sentence[max(i - window, 0): i]
            after = sentence[i + 1: i + window + 1]

            center_and_context["centre"].append(sentence[i])
            center_and_context["context"].append(" ".join([w for l in [before, after] for w in l]))
    
    df = pd.DataFrame(center_and_context)           
    return df
        
foo = corpus_reader(data_path, WINDOW_SIZE)
foo.to_csv("training_data.csv", sep="\t", index=False)

We sampled 50 000 senteces completely random from the *whole* wikipedia for our training data. Give some reasons why this is good, and why it might be bad. (*note*: We'll have a few questions like these, one or two reasons for and against is sufficient)

[2 marks]

Pros: It gives a fairly wide coverage of different types of sentences with different topics, words, etc.
Cons: It might be too general ??? skasdoaksdoaps

In [3]:

from torchtext.legacy.data import Field, BucketIterator, Iterator, TabularDataset


### Loading the data

We now need to load the data in an appropriate format for torchtext (https://torchtext.readthedocs.io/en/latest/). We'll use PyText for this and it'll follow the same structure as I showed you in the lecture (remember to lower-case all tokens). Create a function which returns a (bucket)iterator of the training data, and the vocabulary object (```Field```). 

(*hint1*: you can format the data such that the center word always is first, then you only need to use one field)

(*hint2*: the code I showed you during the leture is available in /files/pytorch_tutorial/ on canvas)

[4 marks]

In [4]:

def get_data():
    ddir = 'training_data.csv'
    whitespacer = lambda x: x.split(' ')

    TOKENS = Field(
        tokenize=whitespacer,
        lower=True,                   
        batch_first=True
    ) 

    train = TabularDataset(
        path=ddir,                 
        format='csv',                                        
        fields=[
            ('target', TOKENS),
            ('context', TOKENS),
        ],                                        
        skip_header=True,                                        
        csv_reader_params={
            'delimiter':'\t',                                                             
            'quotechar':'½',
        }
    )

    TOKENS.build_vocab(train)
    
    train_iter = BucketIterator(
        train,                                                  
        batch_size=16,
        sort_within_batch=True,                                                  
        sort_key=lambda x: len(x.target),                                                  
        shuffle=True,                                                  
        device=device
    )   
    
    return train_iter, TOKENS

In [5]:
center, context = get_data()

We lower-cased all tokens above; give some reasons why this is a good idea, and why it may be harmful to our embeddings.

[2 marks]

a capital letter might indicate the beginning of a sentence or a pronoun

## Word Embeddings Model

We will implement the CBOW model for constructing word embedding models.

In [6]:
import torch.optim as optim

In the CBOW model we try to predict the center word based on the context. That is, we take as input ```n``` context words, encode them as vectors, then combine them by summation. This will give us one embedding. We then use this embedding to predict *which* word in our vocabuary is the most likely center word. 

Implement this model 

[7 marks]

In [7]:
class CBOWModel(nn.Module):
    def __init__(self, n_words, emb_dim):
        super(CBOWModel, self).__init__()
        self.embeddings = nn.Embedding(n_words, emb_dim)
        self.prediction = nn.Linear(emb_dim, n_words)
    
    def forward(self, context):
        embedded_context = self.embeddings(context)
        projection = self.projection_function(embedded_context)
        predictions = self.prediction(projection)

        return predictions
        
    def projection_function(self, xs):
        """
        This function will take as input a tensor of size (B, S, D)
        where B is the batch_size, S the window size, and D the dimensionality of embeddings
        this function should compute the sum over the embedding dimensions of the input, 
        that is, we transform (B, S, D) to (B, 1, D) or (B, D) 
        """
        xs_sum = torch.sum(xs, dim=-2)
        return xs_sum

Now we need to train the models. First we define which hyperparameters to use. (You can change these, for example when *developing* your model you can use a batch size of 2 and a very low dimensionality (say 10), just to speed things up). When actually training your model *fo real*, you can use a batch size of [8,16,32,64], and embedding dimensionality of [128,256].

In [8]:
# you can change these numbers to suit your needs :)
dataset, vocab = get_data()
word_embeddings_hyperparameters = {'epochs':3,
                                   'batch_size':16,
                                   'embedding_size':128,
                                   'learning_rate':0.001,
                                   'embedding_dim':128}

Train your model. Iterate over the dataset, get outputs from your model, calculate loss and backpropagate.

We mentioned in the lecture that we use Negative Log Likelihood (https://pytorch.org/docs/stable/generated/torch.nn.NLLLoss.html) loss to train Word2Vec model. In this lab we'll take a shortcut when *training* and use Cross Entropy Loss (https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html), basically it combines ```log_softmax``` and ```NLLLoss```. So what your model should output is a *score* for each word in our vocabulary. The ```CrossEntropyLoss``` will then assign probabilities and calculate the negative log likelihood loss.

[3 marks]

In [9]:
# load data
dataset, vocab = get_data()

# build model and construct loss/optimizer
cbow_model = CBOWModel( 
    n_words=len(vocab.vocab),
    emb_dim=word_embeddings_hyperparameters['embedding_dim']
)

cbow_model.to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(cbow_model.parameters(), lr=word_embeddings_hyperparameters['learning_rate'])


In [12]:

# start training loop
total_loss = 0
for epoch in range(word_embeddings_hyperparameters['epochs']):
    for i, batch in enumerate(dataset):
        
        context = batch.context
        target_word = batch.target
        
        # send your batch of sentences to the model
        output = cbow_model(context)
        
        # compute the loss, you'll need to reshape the input
        # you can read more about this is the documentation for
        # CrossEntropyLoss
        loss = loss_fn(output, target_word.reshape(-1))
        total_loss += loss.item()
        
        # print average loss for the epoch
        print(total_loss/(i+1), end='\r') 
        
        # compute gradients
        loss.backward()
        
        # update parameters
        optimizer.step()
        
        # reset gradients
        optimizer.zero_grad()
    print()
        

7.7794695569048475
13.792637217209614
19.093325332200045


In [13]:
torch.save(cbow_model,"model")

In [10]:
cbow_model = torch.load("model")

In [11]:
word_to_ix = {"hello": 0, "world": 1}
embeds = cbow_model.embeddings  # 2 words in vocab, 5 dimensional embeddings
lookup_tensor = torch.tensor([word_to_ix["hello"]], dtype=torch.long)
hello_embed = embeds(lookup_tensor)
print(hello_embed)

tensor([[ 0.4616, -0.4370,  0.6094, -2.5003, -0.0399, -0.6730,  0.8233, -1.9801,
          0.2130, -0.6725,  1.4609, -1.6610,  0.5300, -0.3958, -0.2150, -0.4479,
          1.3769,  0.7278,  0.3227,  0.2877, -1.2614,  0.3234, -0.2809,  0.7944,
          0.4774, -0.2587, -1.0860, -0.0977, -0.5257,  0.7849, -1.9698,  1.9914,
         -0.6868, -1.4798, -1.1967,  0.0049,  0.3359,  0.4941, -1.4552, -2.2216,
          0.2248,  0.5556,  0.0345,  1.4029,  0.9312,  1.9038,  1.5891,  0.1998,
         -1.2094,  0.1044,  0.8550,  0.5072,  0.5406, -0.3947,  0.4393,  1.0943,
          0.0820, -0.7398,  1.4119, -0.8577,  2.0025,  0.7836, -0.7303,  1.0984,
          0.8274, -0.1341, -1.0215,  0.0312, -0.4355, -0.7348, -1.0244,  1.3677,
         -0.0763,  1.1809, -1.3128, -0.7483, -1.8014,  0.3068,  0.8211, -1.7521,
          0.7463,  0.8091, -1.4162, -0.4463, -1.1359,  0.2685, -0.6925,  1.4683,
         -0.5736, -0.2304, -0.5217,  0.8267,  0.3177, -1.5126, -1.1772,  0.0508,
          0.4084, -0.2804, -

## Evaluating the model

We will evaluate the model on a dataset of word similarities, WordSim353 (http://alfonseca.org/eng/research/wordsim353.html , also avalable in vanvas under files/03-l). The first thing we need to do is read the dataset and translate it to integers. What we'll do is to reuse the ```Field``` that records word indexes (the second output of ```get_data()```) and use it to parse the file.

The wordsim data is structured as follows:

```
word1 word2 score
...
```


The ```Field``` we got from ```read_data()``` has two built-in functions, ```stoi``` which maps a string to an integer and ```itos``` which maps an integer to a string. 

What our datareader needs to do is: 

```
for line in file:
    word1, word2, score = file.split()
    # encode word1 and word2 as integers
    word1_idx = vocab.vocab.stoi[word1]
    word2_idx = vocab.vocab.stoi[word2]
```

when we have the integers for ```word_1``` and ```word2``` we'll compute the similarity between their word embeddings with *cosine simlarity*. We can obtain the embeddings by querying the embedding layer of the model.

We calculate the cosine similarity for each word pair in the dataset, then compute the pearson correlation between the similarities we obtained with the scores given in the dataset. 

[4 marks]

In [13]:
def read_wordsim(path, vocab, embeddings):
    dataset_sims = []
    model_sims = []
    words = []
    with open(path) as f:
        for line in f:
            word1, word2, score = line.split()

            score = float(score)
            dataset_sims.append(score)
            
            # get the index for the word
            word1_idx = vocab.vocab.stoi[word1]
            word2_idx = vocab.vocab.stoi[word2]

            # get the embedding of the word
            word1_emb = embeddings(torch.LongTensor([word1_idx]))
            word2_emb = embeddings(torch.LongTensor([word2_idx]))
            
            # compute cosine similarity, we'll use the version included in pytorch functional
            # https://pytorch.org/docs/master/generated/torch.nn.functional.cosine_similarity.html
            cosine_similarity = F.cosine_similarity(word1_emb,word2_emb)
            
            x = word1, word2
            
            words.append(x)
            
            model_sims.append(cosine_similarity.item())
    
    return dataset_sims, model_sims, words

path = 'wordsim_similarity_goldstandard.txt'
data, model, words = read_wordsim(path, vocab, cbow_model.embeddings)
pearson_correlation = np.corrcoef(data, model)
            
# the non-diagonals give the pearson correlation,
print(pearson_correlation)

abc = [(x+1)*5 for x in model]

[[1.         0.26582633]
 [0.26582633 1.        ]]


The first value is a comparison between wordsim and itself and the second value is a comparsion between wordsim and our model.
For the second row the values are reversed.

0 = no correlation
1 = perfect correlation

There seems to be a weak correlation between our model and the wordsim model (0.22).

######################try it with more sentences############################

Do you think the model performs good or bad? Why?

[3 marks]

In [15]:
model

# you can compute the difference between data (which contains gold standard similarity)

difference = abs(np.array(data) - np.array(abc))

# print(difference[torch.topk(torch.from_numpy(-difference), 10)[1].numpy()])

# ordered = sorted(difference)
# print(ordered[:10])
# print(ordered[-10:])

from operator import itemgetter
from heapq import nlargest
from heapq import nsmallest

best = nsmallest(10, enumerate(difference), itemgetter(1))

worst = nlargest(10, enumerate(difference), itemgetter(1))

best_idx = []
for item in best:
    index = item[0]
    best_idx.append(index)

worst_idx = []
for item in worst:
    index = item[0]
    worst_idx.append(index)
    
print("Best indices: ", best_idx)
print("Worst indices: ", worst_idx)
    
with open('wordsim_similarity_goldstandard.txt', "r") as f:
    gold_standard = f.read()
    
gold_tuples = [tuple(row.split('\t')) for row in gold_standard.split('\n')]

print("The best performances: \n")
for index in best_idx:
    word_idx = index
    print("")
    print(vocab.vocab.stoi[gold_tuples[word_idx][0]], vocab.vocab.stoi[gold_tuples[word_idx][1]])
    print(gold_tuples[index])
    
print("The worst performances: \n")
for index in worst_idx:
    word_idx = index
    print(vocab.vocab.stoi[gold_tuples[word_idx][0]], vocab.vocab.stoi[gold_tuples[word_idx][1]])
    print(gold_tuples[index])

Best indices:  [1, 110, 71, 109, 103, 91, 107, 112, 114, 104]
Worst indices:  [196, 21, 202, 199, 17, 193, 28, 74, 197, 18]
The best performances: 


0 0
('tiger', 'tiger', '10.00')

2665 388
('ministry', 'culture', '4.69')

1917 1145
('travel', 'activity', '5.00')

1657 525
('consumer', 'energy', '4.75')

1963 580
('street', 'children', '4.94')

479 1269
('man', 'governor', '5.25')

385 7460
('word', 'similarity', '4.75')

4693 972
('investigation', 'effort', '4.59')

147 209
('life', 'term', '4.50')

1507 805
('car', 'flight', '4.94')
The worst performances: 

0 0
('lad', 'wizard', '0.92')
0 0
('Arafat', 'Jackson', '2.50')
210 0
('king', 'cabbage', '0.23')
0 1911
('noon', 'string', '0.54')
0 1540
('fuck', 'sex', '9.44')
2099 147
('stock', 'life', '0.92')
4702 18000
('journey', 'voyage', '9.29')
1963 7860
('street', 'avenue', '8.88')
3168 1027
('sugar', 'approach', '0.88')
559 16868
('football', 'soccer', '9.03')


Select the 10 best and 10 worst performing word pairs, can you see any patterns that explain why *these* are the best and worst word pairs?

[3 marks]

Its good that "tiger" and "tiger" is the most similar word pair since it is comparing the same vector with itself.
However, the other words also seem quite similar, even the bottom words. It is not obvious what causes "asylum" and "madhouse" to be similar that does not carry over to make "liquid" and "water" similar. 

Suggest some ways of improving the model we apply to WordSim353.

[3 marks]

Increase the number of epochs would probably make the model better
Way more data

More data! 
More epochs?
Batch size? 
A larger embedding size?
Smaller learning rate?
Larger embedding dimension?

If we consider a scenario where we use these embeddings in a downstream task, for example sentiment analysis (roughly: determining whether a sentence is positive or negative). 

Give some examples why the sentiment analysis model would benefit from our embeddnings and one examples why our embeddings could hur the performance of the sentiment model.

[3 marks]

Our model does not distinquish between "smart" and "stupid" and that cant be good for a sentiment analysis.

It does not seem to be very fine grained in the way that it distinguishes between synonyms and antonyms.

# Language modeling

In this second part we'll build a simple LSTM language model. Your task is to construct a model which takes a sentence as input and predict the next word for each word in the sentence. For this you'll use the ```LSTM``` class provided by PyTorch (https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html). You can read more about the LSTM here: https://colah.github.io/posts/2015-08-Understanding-LSTMs/

NOTE!!!: Use the same dataset (wiki-cropus.txt) as before.

Our setup is similar to before, we first encode the words as distributed representations then pass these to the LSTM and for each output we predict the next word.

For this we'll build a new dataloader with torchtext, the file we pass to the dataloader should contain one sentence per line, with words separated by whitespace.

```
word_1, ..., word_n
word_1, ..., word_k
...
```

in this dataloader you want to make sure that each sentence begins with a ```<start>``` token and ends with a ```<end>``` token, there is a keyword argument in ```Field``` for this :). But other than that, as before you read the dataset and output a iterator over the dataset and a vocabulary. 

Implement the dataloader, language model and the training loop (the training loop will basically be the same as for word2vec).

[12 marks]

In [24]:
# you can change these numbers to suit your needs as before :)
lm_hyperparameters = {'epochs':3,
                      'batch_size':16,
                      'learning_rate':0.001,
                      'embedding_dim':128,
                      'output_dim':128}

In [33]:
counter = 0
newfile = open("small_corpus.txt", "x")
with open("wiki-corpus.txt") as f:
    for line in f:
        if counter < 5000:
            newfile.write(line)
            counter += 1


FileExistsError: [Errno 17] File exists: 'small_corpus.txt'

In [34]:
data_path = 'small_corpus.txt'

def get_data():
    whitespacer = lambda x: x.split(' ')

    TOKENS = Field(
        tokenize=whitespacer,
        lower=True,                   
        batch_first=True,
        init_token='<start>',
        eos_token='<end>',
    )
    
    train = TabularDataset(
        path=data_path,                 
        format='csv',                                        
        fields=[('sentence', TOKENS)],
        skip_header=False,                                        
        csv_reader_params={
            'delimiter':'\n',                                                             
            'quotechar':'½',
        }
    )

    TOKENS.build_vocab(train)
    
    train_iter = BucketIterator(
        train,                                                  
        batch_size=lm_hyperparameters['batch_size'],
        sort_within_batch=True,                                                  
        sort_key=lambda x: len(x.sentence),                                                  
        shuffle=True,                                                  
        device=device
    )   
    

    return train_iter, TOKENS

In [43]:
class LM_withLSTM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, output_dim):
        super(LM_withLSTM, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.LSTM = nn.LSTM(embedding_dim, output_dim)
        self.predict_word = nn.Linear(output_dim, vocab_size)
    
    def forward(self, seq):
        embedded_seq = self.embeddings(seq)
        timestep_representation, *_ = self.LSTM(embedded_seq)
        predicted_words = self.predict_word(timestep_representation)
        
        return predicted_words

In [44]:
# load data
dataset, vocab = get_data()

# build model and construct loss/optimizer
lm_model = LM_withLSTM(len(vocab.vocab), 
                       lm_hyperparameters['embedding_dim'],
                       lm_hyperparameters['output_dim'])
lm_model.to(device)

loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(cbow_model.parameters(), lr=lm_hyperparameters['learning_rate'])

# start training loop
total_loss = 0
for epoch in range(lm_hyperparameters['epochs']):
    for i, batch in enumerate(dataset):
        
        # the strucure for each BATCH is:
        # <start>, w0, ..., wn, <end>
        sentence = batch.sentence
        
        # when training the model, at each input we predict the *NEXT* token
        # consequently there is nothing to predict when we give the model 
        # <end> as input. 
        # thus, we do not want to give <end> as input to the model, select 
        # from each batch all tokens except the last. 
        # tip: use pytorch indexing/slicing (same as numpy) 
        # (https://pytorch.org/tutorials/beginner/basics/tensorqs_tutorial.html#operations-on-tensors)
        # (https://jhui.github.io/2018/02/09/PyTorch-Basic-operations/)
        input_sentence = sentence[:, :-1]
        
        # send your batch of sentences to the model
        output = lm_model(input_sentence)
        
        # for each output, the model predict the NEXT token, so we have to reshape 
        # our dataset again. On timestep t, we evaluate on token t+1. That is,
        # we never predict the <start> token ;) so this time, we select all but the first 
        # token from sentences (that is, all the tokens that we predict)
        gold_data = sentence[:, 1:]
        
        # the shape of the output and sentence variable need to be changed,
        # for the loss function. Details are in the documentation.
        # You can use .view(...,...) to reshape the tensors  
        loss = loss_fn(output.view(output.shape[0], -1, output.shape[1]), gold_data)
        total_loss += loss.item()
        
        # print average loss for the epoch
        print(total_loss/(i+1), end='\r') 
        
        # compute gradients
        loss.backward()
        
        # update parameters
        optimizer.step()
        
        # reset gradients
        optimizer.zero_grad()
    print()

9.928844811839442
19.857567774864936
29.786478374850365


In [46]:
torch.save(lm_model,"model_lstm")

In [47]:
lstm_model = torch.load("model_lstm")

In [85]:
lstm_model

LM_withLSTM(
  (embeddings): Embedding(20379, 128)
  (LSTM): LSTM(128, 128)
  (predict_word): Linear(in_features=128, out_features=20379, bias=True)
)

### Evaluating the language model

We'll evaluate our model using the BLiMP dataset (https://github.com/alexwarstadt/blimp). The BLiMP dataset contains sets of linguistic minimal pairs for various syntactic and semantic phenomena, We'll evaluate our model on *existential quantifiers* (link: https://github.com/alexwarstadt/blimp/blob/master/data/existential_there_quantifiers_1.jsonl). This data, as the name suggests, investigate whether language models assign higher probability to *correct* usage of there-quantifiers. 

An example entry in the dataset is: 

```
{"sentence_good": "There was a documentary about music irritating Allison.", "sentence_bad": "There was each documentary about music irritating Allison.", "field": "semantics", "linguistics_term": "quantifiers", "UID": "existential_there_quantifiers_1", "simple_LM_method": true, "one_prefix_method": false, "two_prefix_method": false, "lexically_identical": false, "pairID": "0"}
```

Download the dataset and build a datareader (similar to what you did for word2vec). The dataset structure you should aim for is (you don't need to worry about the other keys for this assignment):

```
good_sentence_1, bad_sentence_1
...
```

your task now is to compare the probability assigned to the good sentence with to the probability assigned to the bad sentence. To compute a probability for a sentence we consider the product of the probabilities assigned to the *gold* tokens, remember, at timestep ```t``` we're predicting which token comes *next* e.g. ```t+1``` (basically, you do the same thing as you did when training).

In rough pseudo code what your code should do is:

```
accuracy = []
for good_sentence, bad_sentence in dataset:
    gs_lm_output = LanguageModel(good_sentence)
    gs_token_probabilities = softmax(gs_lm_output)
    gs_sentence_probability = product(gs_token_probabilities[GOLD_TOKENS])

    bs_lm_output = LanguageModel(bad_sentence)
    bs_token_probabilities = softmax(bs_lm_output)
    bs_sentence_probability = product(bs_token_probabilities[GOLD_TOKENS])

    # int(True) = 1 and int(False) = 0
    is_correct = int(gs_sentence_probability > bs_sentence_probability)
    accuracy.append(is_correct)

print(numpy.mean(accuracy))
    
```

[6 marks]

In [144]:
# your code goes here
import json
from nltk.tokenize import WhitespaceTokenizer
tk = WhitespaceTokenizer()

dataset, vocab = get_data()

def evaluate_model(path, vocab, model):
    
    accuracy = []
    with open(path) as f:
        # iterate over one pair of sentences at a time
        for line in f:
            # load the data
            data = json.loads(line)
            good_s = data['sentence_good']
            bad_s = data['sentence_bad']
            
            
            # the data is tokenized as whitespace
            tok_good_s = tk.tokenize(good_s)
            tok_bad_s = tk.tokenize(bad_s)
            
            
            
            
            # encode your words as integers using the vocab from the dataloader, size is (S)
            # we use unsqueeze to create the batch dimension 
            # in this case our input is only ONE batch, so the size of the tensor becomes: 
            # (S) -> (1, S) as the model expects batches
            
            
            #words are integers in vocab, 0 if not found
            #this is words from the json file now each sentence is a tensor, each word is mapped to an integer
            enc_good_s = torch.tensor([vocab.vocab.stoi.get(x, 0) for x in tok_good_s], device=device).unsqueeze(0) 
            enc_bad_s = torch.tensor([vocab.vocab.stoi.get(x, 0) for x in tok_bad_s], device=device).unsqueeze(0)
            
            
            # pass your encoded sentences to the model and predict the next tokens
            good_s = model(enc_good_s)
            bad_s = model(enc_bad_s)
            
            
            # get probabilities with softmax
            gs_probs = F.softmax(good_s, dim=1)
            bs_probs = F.softmax(bad_s, dim=1)
            
            
            
            # select the probability of the gold tokens
            gs_sent_prob = find_token_probs(gs_probs, enc_good_s)
            bs_sent_prob = find_token_probs(bs_probs, enc_bad_s)
            
            #print(gs_sent_prob)
            
          #  accuracy.append(int(gs_sent_prob>bs_sent_prob))
            
    return 0 #accuracy
            
def find_token_probs(model_probs, encoded_sentence):
    probs = []

    # iterate over the tokens in your encoded sentence
    for token, gold_token in enumerate(encoded_sentence):       #each token is a tensor of numbers corresponding to words in model
        #removed the last token
        gold_token = gold_token[:-1]
        
        output = lstm_model(input_sentence)
        print(output)
        
        
        #get probability for output below...
        
        #???
        
        
        # select the probability of the gold tokens and save
        # hint: pytorch indexing is helpful here ;)
        
       # prob = torch.prod(gold_token)
        #print(prob)
        #print(prob)
        #probs.append(prob)
    #sentence_prob = ...
    return 0 #sentence_prob

path = 'existential_there_quantifiers_1.jsonl'
accuracy = evaluate_model(path, vocab, lstm_model)

print('Final accuracy:')
print(np.round(np.mean(accuracy), 3))


tensor([[[-0.0464, -0.0014, -0.0309,  ...,  0.1471,  0.0708, -0.0298],
         [ 0.0106, -0.0199, -0.0813,  ...,  0.1502,  0.0597,  0.0094],
         [ 0.0243,  0.0280, -0.0314,  ...,  0.0368, -0.0264, -0.0957],
         ...,
         [ 0.0469, -0.1213, -0.0782,  ...,  0.1124,  0.0014,  0.0107],
         [-0.0549,  0.1269, -0.0414,  ...,  0.0360,  0.1417,  0.0481],
         [-0.0344, -0.0485, -0.0553,  ...,  0.0953, -0.0089, -0.0456]],

        [[-0.0323,  0.0381, -0.0290,  ...,  0.1993,  0.0824, -0.0302],
         [ 0.0774,  0.0295, -0.1280,  ...,  0.2076,  0.0982, -0.0109],
         [ 0.0096,  0.0285, -0.1254,  ...,  0.1248, -0.0832, -0.1149],
         ...,
         [ 0.0472, -0.1248, -0.0433,  ...,  0.1264,  0.0502,  0.0318],
         [-0.0137,  0.0161, -0.0116,  ..., -0.1443,  0.0923,  0.0674],
         [-0.0193, -0.0419, -0.0626,  ...,  0.1031, -0.0160, -0.0438]],

        [[-0.0153,  0.0610, -0.0257,  ...,  0.2324,  0.0829, -0.0352],
         [ 0.0740,  0.0566, -0.1502,  ...,  0

IndexError: too many indices for tensor of dimension 1

### Analysis

Our model get some score, say, 55% correct predictions. Is this good? Suggest some *baseline* (i.e. a stupid "model" we hope ours is better than) we can compare the model against.

[3 marks]

Suggest some improvements you could make to your language model.

[3 marks]

Suggest some other metrics we can use to evaluate our system

[2 marks]

# Literature


Neural architectures:
* Y. Bengio, R. Ducharme, P. Vincent, and C. Janvin. A neural probabilistic language model. (Links to an external site.) Journal of Machine Learning Research, 3(6):1137–1155, 2003. (Sections 3 and 4 are less relevant today and hence you can glance through them quickly. Instead, look at the Mikolov papers where they describe training word embeddings with the current neural network architectures.)
* T. Mikolov, K. Chen, G. Corrado, and J. Dean. Efficient estimation of word representations in vector space. arXiv preprint arXiv:1301.3781, 2013.
* T. Mikolov, I. Sutskever, K. Chen, G. S. Corrado, and J. Dean. Distributed representations of words and phrases and their compositionality. In Advances in neural information processing systems, pages 3111–3119, 2013.
    


Total marks: 63